In [3]:
import tkinter as tk
from tkinter import messagebox
from bs4 import BeautifulSoup

HTML_FILE = 'index.html'

class HTMLSectionEditor:
    def __init__(self, root):
        self.root = root
        self.root.title('HTML Editor')

        self.soup = self.load_html()
        self.section_map = self.get_section_ids_and_titles()

        self.current_section_id = None
        self.section_buttons = {}

        # Get and store the current name for dynamic UI label
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        self.name_var = tk.StringVar()
        self.name_var.set(f"Edit Name ({left_name_tag.get_text(strip=True)})" if left_name_tag else "Edit Name")

        self.build_layout()

        # No text and no border initially
        self.text_display.config(state="normal")
        self.text_display.delete("1.0", tk.END)
        self.text_display.config(state="disabled")


    def load_html(self):
        with open(HTML_FILE, 'r', encoding='utf-8') as file:
            return BeautifulSoup(file, 'html.parser')

        
    def save_html(self):
        with open(HTML_FILE, 'w', encoding='utf-8') as file:
            file.write(str(self.soup))

            
    def get_section_ids_and_titles(self):
        sections = self.soup.find_all('h3')
        section_map = {}
        for sec in sections:
            if 'id' in sec.attrs:
                section_map[sec['id']] = sec.text.strip()
        return section_map

    
    def build_layout(self):
        self.root.geometry('1200x600')

        # Left panel
        self.left_frame = tk.Frame(self.root, width=200, bg='#f0f0f0')
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y)

        # Divider
        self.divider = tk.Frame(self.root, width=2, bg='#d3d3d3')
        self.divider.pack(side=tk.LEFT, fill=tk.Y)

        # Right panel
        self.right_frame = tk.Frame(self.root, padx=10, pady=10)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # Section title label
        self.section_title_label = tk.Label(self.right_frame, text='', font=('Arial', 14, 'bold'))
        self.section_title_label.pack(pady=(0, 10))

        # Text display
        self.text_display = tk.Text(
            self.right_frame,
            wrap=tk.WORD,
            height=15,
            state='disabled',
            bg=self.right_frame.cget('bg'),
            relief='flat',
            bd=0
        )
        self.text_display.pack(fill=tk.BOTH, expand=True)

        # Button panel (right side)
        self.button_frame = tk.Frame(self.right_frame)
        self.button_frame.pack(pady=5)

        # Initially hide the buttons
        self.delete_button = tk.Button(self.button_frame, text='Delete', command=self.delete_section, bg='red', fg='white')
        self.delete_button.pack(side=tk.LEFT, padx=5)
        self.delete_button.pack_forget()  # Hide initially

        self.edit_button = tk.Button(self.button_frame, text='Edit', command=self.enable_editing)
        self.edit_button.pack(side=tk.LEFT, padx=5)
        self.edit_button.pack_forget()  # Hide initially

        self.save_button = tk.Button(self.button_frame, text='Save', command=self.save_changes)
        self.save_button.pack(side=tk.LEFT, padx=5)
        self.save_button.pack_forget()  # Hide initially

        # Cancel button that will appear when editing
        self.cancel_button = tk.Button(self.button_frame, text='Cancel', command=self.cancel_editing)
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        self.cancel_button.pack_forget()  # Hide initially

        # Add "Edit Name" tab to left panel
        self.name_btn = tk.Button(
            self.left_frame,
            textvariable=self.name_var,
            width=25,
            anchor='w',
            command=self.edit_name_clicked
        )
        self.name_btn.pack(pady=(10, 2), padx=5)

        # Add section buttons
        for sec_id, title in self.section_map.items():
            btn = tk.Button(
                self.left_frame,
                text=title,
                width=25,
                anchor='w',
                command=lambda sid=sec_id: self.show_section(sid)
            )
            btn.pack(pady=2, padx=5)
            self.section_buttons[sec_id] = btn

        # "+ Add Section" and "Close" at bottom
        self.add_section_form()


    def show_section(self, section_id):
        self.current_section_id = section_id
        self.soup = self.load_html()  # Reload fresh content from file
        section = self.soup.find("h3", {"id": section_id})

        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                # Update the section title label
                section_title = self.section_map.get(section_id, section_id)
                self.section_title_label.config(text=f"Editing: {section_title}")

                # Update the text display
                self.text_display.config(
                    state='normal',
                    bg='white',
                    relief='sunken',
                    bd=1
                )
                self.text_display.delete("1.0", tk.END)
                self.text_display.insert(tk.END, next_p.get_text())
                self.text_display.config(state='disabled')

                # Show Edit, Delete, Save, and Cancel buttons
                self.delete_button.pack(side=tk.LEFT, padx=5)  # Show delete button
                self.edit_button.pack(side=tk.LEFT, padx=5)  # Show edit button
                self.save_button.pack_forget()  # Hide Save initially
                self.cancel_button.pack_forget()  # Hide Cancel initially

                # Highlight the clicked section button and reset others
                for sid, btn in self.section_buttons.items():
                    btn.config(bg='#f0f0f0', fg='black')
                self.highlight_button(self.section_buttons.get(section_id))

            else:
                messagebox.showerror("Error", f"No paragraph found after section '{section_id}'.")
        else:
            messagebox.showerror("Error", f"Section ID '{section_id}' not found.")


    def enable_editing(self):
        if self.current_section_id:
            self.text_display.config(state='normal')
            self.save_button.pack(side=tk.LEFT, padx=5)  # Show save button
            self.cancel_button.pack(side=tk.LEFT, padx=5)  # Show cancel button

            
    def cancel_editing(self):
        # Restore original content and hide the Save and Cancel buttons
        section = self.soup.find('h3', {'id': self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                # Reset the text display content to its original state
                self.text_display.config(state='normal')
                self.text_display.delete('1.0', tk.END)
                self.text_display.insert(tk.END, next_p.get_text())
                self.text_display.config(state='disabled')

        # Hide Save and Cancel buttons
        self.save_button.pack_forget()
        self.cancel_button.pack_forget()


    def save_changes(self):
        new_text = self.text_display.get('1.0', tk.END).strip()
        section = self.soup.find('h3', {'id': self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                next_p.string = new_text
                self.save_html()
                self.text_display.config(state="disabled")
                self.save_button.pack_forget()
                messagebox.showinfo('Success', f"'{self.current_section_id}' updated successfully.")
            else:
                messagebox.showerror("Error", 'Could not find paragraph to update.')

    
    # Method to handle the "Edit Name" tab being clicked
    def edit_name_clicked(self):
        # Highlight the "Edit Name" button
        self.highlight_button(self.name_btn)

        # Open the "Edit Name" popup
        self.edit_name_popup()

        
    # Method to highlight the button when clicked
    def highlight_button(self, button):
        # Reset all section buttons' background color to default
        for btn in self.section_buttons.values():
            btn.config(bg='#f0f0f0', fg='black')

        # Reset the "Edit Name" button background color
        self.name_btn.config(bg='#f0f0f0', fg='black')

        # If a valid button is passed, highlight it
        if button:
            button.config(bg='#4CAF50', fg='white')


    def edit_name_popup(self):
        """
        Open the popup to edit the user's name.
        """
        popup = tk.Toplevel()
        popup.title("Edit Name")
        popup.geometry("400x200")
        popup.grab_set()

        # Get current name from HTML
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        right_name_tag = self.soup.select_one(".right-panel h3#about")
        current_name = left_name_tag.get_text(strip=True) if left_name_tag else ""

        tk.Label(popup, text="Enter your name:").pack(pady=10)
        name_entry = tk.Entry(popup, width=40)
        name_entry.insert(0, current_name)
        name_entry.pack(pady=5)

        # Save button function
        def on_save():
            new_name = name_entry.get().strip()
            if not new_name:
                messagebox.showwarning("Input Error", "Name cannot be empty.")
                return

            if left_name_tag:
                left_name_tag.string = new_name
            if right_name_tag:
                right_name_tag.string = f"{new_name}'s Personal Site"

            # Update name in the footer (example: assuming the footer has an id 'footer-name')
            footer_name_tag = self.soup.select_one("#footer-name")  # Adjust selector based on actual HTML
            if footer_name_tag:
                footer_name_tag.string = new_name

            self.save_html()

            # Update button label in UI
            self.name_var.set(f"Edit Name ({new_name})")

            # Show confirmation popup after saving
            popup.destroy()

            # Show success message
            messagebox.showinfo("Success", "Name updated successfully.")

            # Remove highlight from the "Edit Name" button after success
            self.highlight_button(None)  # Reset the highlight

        save_button = tk.Button(popup, text="Save", command=on_save, bg="#4CAF50", fg="white")
        save_button.pack(pady=15)

                
    def add_section_form(self):
        form_frame = tk.Frame(self.left_frame, bg='#dcdcdc', pady=10)
        form_frame.pack(side=tk.BOTTOM, fill=tk.X)

        # Add "Add Section" button
        tk.Button(
            form_frame,
            text='Add Section',
            command=self.open_add_section_popup,
            bg='#4CAF50',
            fg='white'
        ).pack(pady=(10, 5))

        # Add "Refresh" button (previously "Close")
        tk.Button(
            form_frame,
            text="Refresh",
            command=self.refresh_interface,
            bg="#4CAF50",
            fg="white"
        ).pack(pady=(0, 10))

        
    # New function to refresh the interface by reloading the HTML
    def refresh_interface(self):
        # Reload HTML content
        self.soup = self.load_html()

        # Rebuild the UI based on the updated HTML
        self.section_map = self.get_section_ids_and_titles()

        # Clear and recreate the section buttons in the left panel
        for btn in self.section_buttons.values():
            btn.destroy()
        self.section_buttons.clear()

        for sec_id, title in self.section_map.items():
            btn = tk.Button(
                self.left_frame,
                text=title,
                width=25,
                anchor="w",
                command=lambda sid=sec_id: self.show_section(sid)
            )
            btn.pack(pady=2, padx=5)
            self.section_buttons[sec_id] = btn

        # Only add "Edit Name" button if it doesn't already exist
        name_btn_exists = any(btn.cget("text") == self.name_var.get() for btn in self.left_frame.winfo_children())

        if not name_btn_exists:
            # Add "Edit Name" button if it doesn't already exist
            name_btn = tk.Button(
                self.left_frame,
                textvariable=self.name_var,
                width=25,
                anchor="w",
                command=self.edit_name_popup
            )
            name_btn.pack(pady=(10, 5), padx=5)

        # Optionally, you can also refresh the window title here if needed
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        current_name = left_name_tag.get_text(strip=True) if left_name_tag else "John Doe"
        self.root.title(f"{current_name}'s Personal Site")

          
    def open_add_section_popup(self):
        popup = tk.Toplevel()
        popup.title('Add New Section')
        popup.grab_set()
        popup.geometry('800x500')
        popup.resizable(True, True)

        tk.Label(popup, text='Section ID:').pack(pady=5)
        id_entry = tk.Entry(popup, width=30)
        id_entry.pack()

        tk.Label(popup, text='Title:').pack(pady=5)
        title_entry = tk.Entry(popup, width=30)
        title_entry.pack()

        tk.Label(popup, text='Content:').pack(pady=5)
        content_entry = tk.Text(popup, height=6, width=40)
        content_entry.pack()

        # Insert position selection
        position_var = tk.StringVar(value="After")
        tk.Label(popup, text="Insert Position:").pack(pady=5)
        position_menu = tk.OptionMenu(popup, position_var, "Before", "After")
        position_menu.pack()

        # Target section dropdown
        target_var = tk.StringVar()
        if self.section_map:
            default_target = list(self.section_map.keys())[0]
            target_var.set(default_target)
        tk.Label(popup, text="Section:").pack(pady=5)
        target_menu = tk.OptionMenu(popup, target_var, *self.section_map.keys())
        target_menu.pack()

        
        def on_add():
            sec_id = id_entry.get().strip()
            title = title_entry.get().strip().upper()
            content = content_entry.get("1.0", tk.END).strip()
            pos = position_var.get()
            target_id = target_var.get()

            if not sec_id or not title or not content:
                messagebox.showwarning("Input Error", "All fields are required.")
                return

            if sec_id in self.section_map:
                messagebox.showerror("Error", f"Section ID '{sec_id}' already exists.")
                return

            right_panel = self.soup.find("div", class_="right-panel")
            left_panel_content = self.soup.select_one(".left-panel .content")

            if not (right_panel and left_panel_content):
                messagebox.showerror("Error", "Unable to find content area to insert section.")
                return

            # Create new HTML elements
            new_h3 = self.soup.new_tag("h3", id=sec_id)
            new_h3.string = title
            new_h3["style"] = "text-align: center; background: #cccccc;"
            new_p = self.soup.new_tag("p")
            new_p.string = content
            new_link = self.soup.new_tag("a", href=f"#{sec_id}")
            new_link.string = title.upper()

            # Insert into HTML
            target_h3 = self.soup.find("h3", {"id": target_id})
            target_link = left_panel_content.find("a", href=f"#{target_id}")

            if target_h3:
                if pos == "Before":
                    target_h3.insert_before(new_p)
                    new_p.insert_before(new_h3)
                else:
                    next_p = target_h3.find_next_sibling("p")
                    if next_p:
                        next_p.insert_after(new_h3)
                        new_h3.insert_after(new_p)
                    else:
                        target_h3.insert_after(new_h3)
                        new_h3.insert_after(new_p)

            if target_link:
                if pos == "Before":
                    target_link.insert_before(new_link)
                else:
                    target_link.insert_after(new_link)

            self.save_html()

            # Refresh UI from updated HTML
            self.section_map = self.get_section_ids_and_titles()

            # Clear and recreate left-side buttons
            for btn in self.section_buttons.values():
                btn.destroy()
            self.section_buttons.clear()

            for sec_id_loop, title_loop in self.section_map.items():
                btn = tk.Button(
                    self.left_frame,
                    text=title_loop,
                    width=25,
                    anchor="w",
                    command=lambda sid=sec_id_loop: self.show_section(sid)
                )
                btn.pack(pady=2, padx=5)
                self.section_buttons[sec_id_loop] = btn

            popup.destroy()
            messagebox.showinfo("Success", f"Section '{title}' added {pos.lower()} '{self.section_map[target_id]}'.")

        tk.Button(popup, text="Add", command=on_add, bg="#4CAF50", fg="white").pack(pady=10)


    def delete_section(self):
        if not self.current_section_id:
            return

        confirm = messagebox.askyesno("Confirm Delete", f"Are you sure you want to delete section '{self.current_section_id}'?")
        if not confirm:
            return

        section = self.soup.find("h3", {"id": self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")

            # Remove section elements
            if next_p:
                next_p.decompose()
            section.decompose()

            # Remove corresponding link in HTML left panel
            left_panel_content = self.soup.select_one(".left-panel .content")
            if left_panel_content:
                nav_links = left_panel_content.find_all("a", href=f"#{self.current_section_id}")
                for link in nav_links:
                    link.decompose()

            self.save_html()

            # Remove from Tkinter UI
            if self.current_section_id in self.section_buttons:
                self.section_buttons[self.current_section_id].destroy()
                del self.section_buttons[self.current_section_id]

            del self.section_map[self.current_section_id]

            # Clear editor
            self.current_section_id = None
            self.section_title_label.config(text='')
            self.text_display.config(state='normal')
            self.text_display.delete("1.0", tk.END)
            self.text_display.config(state='disabled')
            self.edit_button.pack_forget()
            self.save_button.pack_forget()
            self.delete_button.pack_forget()

            messagebox.showinfo("Deleted", "Section deleted successfully.")


    def add_new_section(self):
        sec_id = self.new_id_entry.get().strip()
        title = self.new_title_entry.get().strip()
        content = self.new_content_entry.get().strip()

        if not sec_id or not title or not content:
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return

        if sec_id in self.section_map:
            messagebox.showerror("Error", f"Section ID '{sec_id}' already exists.")
            return

        # Append to the right-panel (find last section)
        right_panel = self.soup.find("div", class_="right-panel")
        if right_panel:
            new_h3 = self.soup.new_tag("h3", id=sec_id)
            new_h3.string = title
            new_h3["style"] = "text-align: center; background: #cccccc;"

            new_p = self.soup.new_tag("p")
            new_p.string = content

            right_panel.append(new_h3)
            right_panel.append(new_p)

            self.save_html()
            messagebox.showinfo("Added", f"Section '{title}' added.")

            # Add to UI
            btn = tk.Button(self.left_frame, text=title, width=25, anchor="w", command=lambda sid=sec_id: self.show_section(sid))
            btn.pack(pady=2, padx=5)

            self.section_map[sec_id] = title

            # Clear form
            self.new_id_entry.delete(0, tk.END)
            self.new_title_entry.delete(0, tk.END)
            self.new_content_entry.delete(0, tk.END)

        else:
            messagebox.showerror("Error", "Could not find container to insert section.")

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = HTMLSectionEditor(root)
    root.mainloop()


In [17]:
import tkinter as tk
from tkinter import messagebox
from bs4 import BeautifulSoup

HTML_FILE = 'index.html'

class HTMLSectionEditor:
    def __init__(self, root):
        self.root = root
        self.root.title('HTML Editor')

        self.soup = self.load_html()
        self.section_map = self.get_section_ids_and_titles()

        self.current_section_id = None
        self.section_buttons = {}

        # Get and store the current name for dynamic UI label
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        self.name_var = tk.StringVar()
        self.name_var.set(f"Edit Name ({left_name_tag.get_text(strip=True)})" if left_name_tag else "Edit Name")

        self.build_layout()

        # No text and no border initially
        self.text_display.config(state="normal")
        self.text_display.delete("1.0", tk.END)
        self.text_display.config(state="disabled")


    def load_html(self):
        with open(HTML_FILE, 'r', encoding='utf-8') as file:
            return BeautifulSoup(file, 'html.parser')

        
    def save_html(self):
        with open(HTML_FILE, 'w', encoding='utf-8') as file:
            file.write(str(self.soup))

            
    def get_section_ids_and_titles(self):
        sections = self.soup.find_all('h3')
        section_map = {}
        for sec in sections:
            if 'id' in sec.attrs:
                section_map[sec['id']] = sec.text.strip()
        return section_map

    
    def build_layout(self):
        self.root.geometry('1200x600')

        # Left panel
        self.left_frame = tk.Frame(self.root, width=200, bg='#f0f0f0')
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y)

        # Divider
        self.divider = tk.Frame(self.root, width=2, bg='#d3d3d3')
        self.divider.pack(side=tk.LEFT, fill=tk.Y)

        # Right panel
        self.right_frame = tk.Frame(self.root, padx=10, pady=10)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # Section title frame (Label + Edit button)
        self.section_title_frame = tk.Frame(self.right_frame)
        self.section_title_frame.pack(pady=(0, 10))

        self.section_title_label = tk.Label(self.section_title_frame, text='', font=('Arial', 14, 'bold'))
        self.section_title_label.pack(side=tk.LEFT)

#         self.edit_section_name_btn = tk.Button(
#             self.section_title_frame,
#             text='🖉',
#             font=('Arial', 12, 'bold'),
#             command=self.edit_section_name,
#             relief='flat',
#             bg=self.right_frame.cget('bg'),
#             activebackground=self.right_frame.cget('bg')
#         )
#         self.edit_section_name_btn.pack(side=tk.LEFT, padx=(5, 0))
#         self.edit_section_name_btn.pack_forget()  # hide initially


        self.edit_section_name_btn = tk.Button(
            self.section_title_frame,
            text='🖉',  # Same icon as before
            font=('Arial', 16, 'bold'),  # Just made font bigger and bold
            command=self.edit_section_name,
            relief='flat',  # Keep it flat, no border
            bg=self.right_frame.cget('bg'),  # Match background
            activebackground=self.right_frame.cget('bg'),  # Match active background too
#             bd=0,  # No border
            highlightthickness=0,  # Remove border highlight
            cursor='hand2'  # Change cursor to hand when hover
        )
        self.edit_section_name_btn.pack(side=tk.LEFT, padx=(5, 0))
        self.edit_section_name_btn.pack_forget()


        # Text display
        self.text_display = tk.Text(
            self.right_frame,
            wrap=tk.WORD,
            height=15,
            state='disabled',
            bg=self.right_frame.cget('bg'),
            relief='flat',
            bd=0
        )
        self.text_display.pack(fill=tk.BOTH, expand=True)

        # Button panel (right side)
        self.button_frame = tk.Frame(self.right_frame)
        self.button_frame.pack(pady=5)

        # Initially hide the buttons
        self.delete_button = tk.Button(self.button_frame, text='Delete', command=self.delete_section, bg='red', fg='white')
        self.delete_button.pack(side=tk.LEFT, padx=5)
        self.delete_button.pack_forget()  # Hide initially

        self.edit_button = tk.Button(self.button_frame, text='Edit', command=self.enable_editing)
        self.edit_button.pack(side=tk.LEFT, padx=5)
        self.edit_button.pack_forget()  # Hide initially

        self.save_button = tk.Button(self.button_frame, text='Save', command=self.save_changes)
        self.save_button.pack(side=tk.LEFT, padx=5)
        self.save_button.pack_forget()  # Hide initially

        # Cancel button that will appear when editing
        self.cancel_button = tk.Button(self.button_frame, text='Cancel', command=self.cancel_editing)
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        self.cancel_button.pack_forget()  # Hide initially

        # Add "Edit Name" tab to left panel
        self.name_btn = tk.Button(
            self.left_frame,
            textvariable=self.name_var,
            width=25,
            anchor='w',
            command=self.edit_name_clicked
        )
        self.name_btn.pack(pady=(10, 2), padx=5)

        # Add section buttons
        for sec_id, title in self.section_map.items():
            btn = tk.Button(
                self.left_frame,
                text=title,
                width=25,
                anchor='w',
                command=lambda sid=sec_id: self.show_section(sid)
            )
            btn.pack(pady=2, padx=5)
            self.section_buttons[sec_id] = btn

        # "+ Add Section" and "Close" at bottom
        self.add_section_form()


    def show_section(self, section_id):
        self.current_section_id = section_id
        self.soup = self.load_html()  # Reload fresh content from file
        section = self.soup.find("h3", {"id": section_id})

        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                # Update the section title label
                section_title = self.section_map.get(section_id, section_id)
                self.section_title_label.config(text=f"Editing: {section_title}")
                self.edit_section_name_btn.pack(side=tk.LEFT, padx=(5, 0))


                # Update the text display
                self.text_display.config(
                    state='normal',
                    bg='white',
                    relief='sunken',
                    bd=1
                )
                self.text_display.delete("1.0", tk.END)
                self.text_display.insert(tk.END, next_p.get_text())
                self.text_display.config(state='disabled')

                # Show Edit, Delete, Save, and Cancel buttons
                self.delete_button.pack(side=tk.LEFT, padx=5)  # Show delete button
                self.edit_button.pack(side=tk.LEFT, padx=5)  # Show edit button
                self.save_button.pack_forget()  # Hide Save initially
                self.cancel_button.pack_forget()  # Hide Cancel initially

                # Highlight the clicked section button and reset others
                for sid, btn in self.section_buttons.items():
                    btn.config(bg='#f0f0f0', fg='black')
                self.highlight_button(self.section_buttons.get(section_id))

            else:
                messagebox.showerror("Error", f"No paragraph found after section '{section_id}'.")
        else:
            messagebox.showerror("Error", f"Section ID '{section_id}' not found.")


    def edit_section_name(self):
        if not self.current_section_id:
            return

        popup = tk.Toplevel()
        popup.title("Edit Section Title")
        popup.geometry("400x200")
        popup.grab_set()

        tk.Label(popup, text="Enter new section title:").pack(pady=10)
        title_entry = tk.Entry(popup, width=40)
        title_entry.insert(0, self.section_map.get(self.current_section_id, ''))
        title_entry.pack(pady=5)

        def on_save():
            new_title = title_entry.get().strip()
            if not new_title:
                messagebox.showwarning("Input Error", "Title cannot be empty.")
                return

            # Generate new ID: lowercase, spaces -> underscores
            new_id = new_title.lower().replace(' ', '_')

            # Find the section <h3> and update
            section_tag = self.soup.find('h3', id=self.current_section_id)
            if section_tag:
                section_tag.string = new_title
                section_tag['id'] = new_id

            # Update the left panel <a> link text and href
            left_link = self.soup.select_one(f'.left-panel .content a[href=\"#{self.current_section_id}\"]')
            if left_link:
                left_link.string = new_title
                left_link['href'] = f'#{new_id}'

            # Save HTML
            self.save_html()

            # Update Python UI
            if self.current_section_id in self.section_buttons:
                btn = self.section_buttons.pop(self.current_section_id)
                btn.config(text=new_title)
                btn.config(command=lambda sid=new_id: self.show_section(sid))
                self.section_buttons[new_id] = btn

            self.section_map.pop(self.current_section_id)
            self.section_map[new_id] = new_title

            self.section_title_label.config(text=f"Editing: {new_title}")

            self.current_section_id = new_id

            popup.destroy()
            messagebox.showinfo("Success", "Section title and ID updated successfully.")

        # 👉 Create Save button **outside** on_save
        save_button = tk.Button(popup, text="Save", command=on_save, bg="#4CAF50", fg="white")
        save_button.pack(pady=15)



    
    def enable_editing(self):
        if self.current_section_id:
            self.text_display.config(state='normal')
            self.save_button.pack(side=tk.LEFT, padx=5)  # Show save button
            self.cancel_button.pack(side=tk.LEFT, padx=5)  # Show cancel button

            
    def cancel_editing(self):
        # Restore original content and hide the Save and Cancel buttons
        section = self.soup.find('h3', {'id': self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                # Reset the text display content to its original state
                self.text_display.config(state='normal')
                self.text_display.delete('1.0', tk.END)
                self.text_display.insert(tk.END, next_p.get_text())
                self.text_display.config(state='disabled')

        # Hide Save and Cancel buttons
        self.save_button.pack_forget()
        self.cancel_button.pack_forget()


    def save_changes(self):
        new_text = self.text_display.get('1.0', tk.END).strip()
        section = self.soup.find('h3', {'id': self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")
            if next_p:
                next_p.string = new_text
                self.save_html()
                self.text_display.config(state="disabled")
                self.save_button.pack_forget()
                messagebox.showinfo('Success', f"'{self.current_section_id}' updated successfully.")
            else:
                messagebox.showerror("Error", 'Could not find paragraph to update.')

    
    # Method to handle the "Edit Name" tab being clicked
    def edit_name_clicked(self):
        # Highlight the "Edit Name" button
        self.highlight_button(self.name_btn)

        # Open the "Edit Name" popup
        self.edit_name_popup()

        
    # Method to highlight the button when clicked
    def highlight_button(self, button):
        # Reset all section buttons' background color to default
        for btn in self.section_buttons.values():
            btn.config(bg='#f0f0f0', fg='black')

        # Reset the "Edit Name" button background color
        self.name_btn.config(bg='#f0f0f0', fg='black')

        # If a valid button is passed, highlight it
        if button:
            button.config(bg='#4CAF50', fg='white')


    def edit_name_popup(self):
        """
        Open the popup to edit the user's name.
        """
        popup = tk.Toplevel()
        popup.title("Edit Name")
        popup.geometry("400x200")
        popup.grab_set()

        # Get current name from HTML
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        right_name_tag = self.soup.select_one(".right-panel h3#about")
        current_name = left_name_tag.get_text(strip=True) if left_name_tag else ""

        tk.Label(popup, text="Enter your name:").pack(pady=10)
        name_entry = tk.Entry(popup, width=40)
        name_entry.insert(0, current_name)
        name_entry.pack(pady=5)

        # Save button function
        def on_save():
            new_name = name_entry.get().strip()
            if not new_name:
                messagebox.showwarning("Input Error", "Name cannot be empty.")
                return

            if left_name_tag:
                left_name_tag.string = new_name
            if right_name_tag:
                right_name_tag.string = f"{new_name}'s Personal Site"

            # Update name in the footer (example: assuming the footer has an id 'footer-name')
            footer_name_tag = self.soup.select_one("#footer-name")  # Adjust selector based on actual HTML
            if footer_name_tag:
                footer_name_tag.string = new_name

            self.save_html()

            # Update button label in UI
            self.name_var.set(f"Edit Name ({new_name})")

            # Show confirmation popup after saving
            popup.destroy()

            # Show success message
            messagebox.showinfo("Success", "Name updated successfully.")

            # Remove highlight from the "Edit Name" button after success
            self.highlight_button(None)  # Reset the highlight

        save_button = tk.Button(popup, text="Save", command=on_save, bg="#4CAF50", fg="white")
        save_button.pack(pady=15)

                
    def add_section_form(self):
        form_frame = tk.Frame(self.left_frame, bg='#dcdcdc', pady=10)
        form_frame.pack(side=tk.BOTTOM, fill=tk.X)

        # Add "Add Section" button
        tk.Button(
            form_frame,
            text='Add Section',
            command=self.open_add_section_popup,
            bg='#4CAF50',
            fg='white'
        ).pack(pady=(10, 5))

        # Add "Refresh" button (previously "Close")
        tk.Button(
            form_frame,
            text="Refresh",
            command=self.refresh_interface,
            bg="#4CAF50",
            fg="white"
        ).pack(pady=(0, 10))

        
    # New function to refresh the interface by reloading the HTML
    def refresh_interface(self):
        # Reload HTML content
        self.soup = self.load_html()

        # Rebuild the UI based on the updated HTML
        self.section_map = self.get_section_ids_and_titles()

        # Clear and recreate the section buttons in the left panel
        for btn in self.section_buttons.values():
            btn.destroy()
        self.section_buttons.clear()

        for sec_id, title in self.section_map.items():
            btn = tk.Button(
                self.left_frame,
                text=title,
                width=25,
                anchor="w",
                command=lambda sid=sec_id: self.show_section(sid)
            )
            btn.pack(pady=2, padx=5)
            self.section_buttons[sec_id] = btn

        # Only add "Edit Name" button if it doesn't already exist
        name_btn_exists = any(btn.cget("text") == self.name_var.get() for btn in self.left_frame.winfo_children())

        if not name_btn_exists:
            # Add "Edit Name" button if it doesn't already exist
            name_btn = tk.Button(
                self.left_frame,
                textvariable=self.name_var,
                width=25,
                anchor="w",
                command=self.edit_name_popup
            )
            name_btn.pack(pady=(10, 5), padx=5)

        # Optionally, you can also refresh the window title here if needed
        left_name_tag = self.soup.select_one(".left-panel .content h3")
        current_name = left_name_tag.get_text(strip=True) if left_name_tag else "John Doe"
        self.root.title(f"{current_name}'s Personal Site")

          
    def open_add_section_popup(self):
        popup = tk.Toplevel()
        popup.title('Add New Section')
        popup.grab_set()
        popup.geometry('800x500')
        popup.resizable(True, True)

        tk.Label(popup, text='Section ID:').pack(pady=5)
        id_entry = tk.Entry(popup, width=30)
        id_entry.pack()

        tk.Label(popup, text='Title:').pack(pady=5)
        title_entry = tk.Entry(popup, width=30)
        title_entry.pack()

        tk.Label(popup, text='Content:').pack(pady=5)
        content_entry = tk.Text(popup, height=6, width=40)
        content_entry.pack()

        # Insert position selection
        position_var = tk.StringVar(value="After")
        tk.Label(popup, text="Insert Position:").pack(pady=5)
        position_menu = tk.OptionMenu(popup, position_var, "Before", "After")
        position_menu.pack()

        # Target section dropdown
        target_var = tk.StringVar()
        if self.section_map:
            default_target = list(self.section_map.keys())[0]
            target_var.set(default_target)
        tk.Label(popup, text="Section:").pack(pady=5)
        target_menu = tk.OptionMenu(popup, target_var, *self.section_map.keys())
        target_menu.pack()

        
        def on_add():
            sec_id = id_entry.get().strip()
            title = title_entry.get().strip().upper()
            content = content_entry.get("1.0", tk.END).strip()
            pos = position_var.get()
            target_id = target_var.get()

            if not sec_id or not title or not content:
                messagebox.showwarning("Input Error", "All fields are required.")
                return

            if sec_id in self.section_map:
                messagebox.showerror("Error", f"Section ID '{sec_id}' already exists.")
                return

            right_panel = self.soup.find("div", class_="right-panel")
            left_panel_content = self.soup.select_one(".left-panel .content")

            if not (right_panel and left_panel_content):
                messagebox.showerror("Error", "Unable to find content area to insert section.")
                return

            # Create new HTML elements
            new_h3 = self.soup.new_tag("h3", id=sec_id)
            new_h3.string = title
            new_h3["style"] = "text-align: center; background: #cccccc;"
            new_p = self.soup.new_tag("p")
            new_p.string = content
            new_link = self.soup.new_tag("a", href=f"#{sec_id}")
            new_link.string = title.upper()

            # Insert into HTML
            target_h3 = self.soup.find("h3", {"id": target_id})
            target_link = left_panel_content.find("a", href=f"#{target_id}")

            if target_h3:
                if pos == "Before":
                    target_h3.insert_before(new_p)
                    new_p.insert_before(new_h3)
                else:
                    next_p = target_h3.find_next_sibling("p")
                    if next_p:
                        next_p.insert_after(new_h3)
                        new_h3.insert_after(new_p)
                    else:
                        target_h3.insert_after(new_h3)
                        new_h3.insert_after(new_p)

            if target_link:
                if pos == "Before":
                    target_link.insert_before(new_link)
                else:
                    target_link.insert_after(new_link)

            self.save_html()

            # Refresh UI from updated HTML
            self.section_map = self.get_section_ids_and_titles()

            # Clear and recreate left-side buttons
            for btn in self.section_buttons.values():
                btn.destroy()
            self.section_buttons.clear()

            for sec_id_loop, title_loop in self.section_map.items():
                btn = tk.Button(
                    self.left_frame,
                    text=title_loop,
                    width=25,
                    anchor="w",
                    command=lambda sid=sec_id_loop: self.show_section(sid)
                )
                btn.pack(pady=2, padx=5)
                self.section_buttons[sec_id_loop] = btn

            popup.destroy()
            messagebox.showinfo("Success", f"Section '{title}' added {pos.lower()} '{self.section_map[target_id]}'.")

        tk.Button(popup, text="Add", command=on_add, bg="#4CAF50", fg="white").pack(pady=10)


    def delete_section(self):
        if not self.current_section_id:
            return

        confirm = messagebox.askyesno("Confirm Delete", f"Are you sure you want to delete section '{self.current_section_id}'?")
        if not confirm:
            return

        section = self.soup.find("h3", {"id": self.current_section_id})
        if section:
            next_p = section.find_next_sibling("p")

            # Remove section elements
            if next_p:
                next_p.decompose()
            section.decompose()

            # Remove corresponding link in HTML left panel
            left_panel_content = self.soup.select_one(".left-panel .content")
            if left_panel_content:
                nav_links = left_panel_content.find_all("a", href=f"#{self.current_section_id}")
                for link in nav_links:
                    link.decompose()

            self.save_html()

            # Remove from Tkinter UI
            if self.current_section_id in self.section_buttons:
                self.section_buttons[self.current_section_id].destroy()
                del self.section_buttons[self.current_section_id]

            del self.section_map[self.current_section_id]

            # Clear editor
            self.current_section_id = None
            self.section_title_label.config(text='')
            self.text_display.config(state='normal')
            self.text_display.delete("1.0", tk.END)
            self.text_display.config(state='disabled')
            self.edit_button.pack_forget()
            self.save_button.pack_forget()
            self.delete_button.pack_forget()

            messagebox.showinfo("Deleted", "Section deleted successfully.")


    def add_new_section(self):
        sec_id = self.new_id_entry.get().strip()
        title = self.new_title_entry.get().strip()
        content = self.new_content_entry.get().strip()

        if not sec_id or not title or not content:
            messagebox.showwarning("Input Error", "Please fill all fields.")
            return

        if sec_id in self.section_map:
            messagebox.showerror("Error", f"Section ID '{sec_id}' already exists.")
            return

        # Append to the right-panel (find last section)
        right_panel = self.soup.find("div", class_="right-panel")
        if right_panel:
            new_h3 = self.soup.new_tag("h3", id=sec_id)
            new_h3.string = title
            new_h3["style"] = "text-align: center; background: #cccccc;"

            new_p = self.soup.new_tag("p")
            new_p.string = content

            right_panel.append(new_h3)
            right_panel.append(new_p)

            self.save_html()
            messagebox.showinfo("Added", f"Section '{title}' added.")

            # Add to UI
            btn = tk.Button(self.left_frame, text=title, width=25, anchor="w", command=lambda sid=sec_id: self.show_section(sid))
            btn.pack(pady=2, padx=5)

            self.section_map[sec_id] = title

            # Clear form
            self.new_id_entry.delete(0, tk.END)
            self.new_title_entry.delete(0, tk.END)
            self.new_content_entry.delete(0, tk.END)

        else:
            messagebox.showerror("Error", "Could not find container to insert section.")

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = HTMLSectionEditor(root)
    root.mainloop()


In [5]:
import json
import re
from typing import List, Dict, Tuple
import nltk
from nltk.tokenize import word_tokenize

# Make sure to have punkt tokenizer
nltk.download('punkt')

# Your label mapping
label2id = {
    'O': 0,
    'B-CHEMICAL': 1,
    'I-CHEMICAL': 2,
    'B-MATERIAL': 3,
    'I-MATERIAL': 4,
    'B-STRUCTURE': 5,
    'I-STRUCTURE': 6,
    'B-PROPERTY': 7,
    'I-PROPERTY': 8,
    'B-APPLICATION': 9,
    'I-APPLICATION': 10,
    'B-PROCESS': 11,
    'I-PROCESS': 12,
    'B-EQUIPMENT': 13,
    'I-EQUIPMENT': 14,
    'B-MEASUREMENT': 15,
    'I-MEASUREMENT': 16,
    'B-ABBREVIATION': 17,
    'I-ABBREVIATION': 18,
}

# Map your NER labels to schema (assumed all go to PROPERTY for this example)
ner_to_schema = {
    'COUNTRY': 'PROPERTY',
    'CAPITAL': 'PROPERTY',
    'CURRENCY': 'PROPERTY'
}

# Input JSON
with open("uu-annotations.json", "r") as f:
    data = json.load(f)

def char_to_token_mapping(text: str, tokens: List[str]) -> List[Tuple[int, int]]:
    mapping = []
    start = 0
    for token in tokens:
        start = text.find(token, start)
        mapping.append((start, start + len(token)))
        start += len(token)
    return mapping

def convert_annotations(data):
    results = []
    for idx, (text, annotation) in enumerate(data["annotations"]):
        tokens = word_tokenize(text)
        mapping = char_to_token_mapping(text, tokens)
        ner_tags = [label2id['O']] * len(tokens)

        for start_char, end_char, entities in annotation["entities"]:
            label = entities[0][3]  # Take the first label
            schema_label = ner_to_schema[label]  # Convert to schema like PROPERTY
            b_label = f'B-{schema_label}'
            i_label = f'I-{schema_label}'
            b_id = label2id[b_label]
            i_id = label2id[i_label]

            # Assign BIO tags based on token positions
            for i, (t_start, t_end) in enumerate(mapping):
                if t_start >= end_char:
                    break
                if t_end <= start_char:
                    continue
                if t_start >= start_char and t_end <= end_char:
                    if ner_tags[i] == label2id['O']:
                        ner_tags[i] = b_id if t_start == start_char else i_id

        results.append({
            "id": str(idx),
            "tokens": tokens,
            "ner_tags": ner_tags
        })

    return results

# Convert and print output
converted = convert_annotations(data)
print(json.dumps(converted, indent=4))


[
    {
        "id": "0",
        "tokens": [
            "Bangladesh",
            "is",
            "a",
            "south",
            "asian",
            "country",
            ".",
            "dhaka",
            "is",
            "its",
            "capital",
            "."
        ],
        "ner_tags": [
            7,
            0,
            0,
            0,
            0,
            0,
            0,
            7,
            0,
            0,
            0,
            0
        ]
    },
    {
        "id": "1",
        "tokens": [
            "its",
            "currency",
            "is",
            "bangladeshi",
            "taka",
            "."
        ],
        "ner_tags": [
            0,
            0,
            0,
            7,
            8,
            0
        ]
    }
]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\umayer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Label to ID mapping
label2id = {
    'O': 0,
    'B-CHEMICAL': 1, 'I-CHEMICAL': 2,
    'B-MATERIAL': 3, 'I-MATERIAL': 4,
    'B-STRUCTURE': 5, 'I-STRUCTURE': 6,
    'B-PROPERTY': 7, 'I-PROPERTY': 8,
    'B-APPLICATION': 9, 'I-APPLICATION': 10,
    'B-PROCESS': 11, 'I-PROCESS': 12,
    'B-EQUIPMENT': 13, 'I-EQUIPMENT': 14,
    'B-MEASUREMENT': 15, 'I-MEASUREMENT': 16,
    'B-ABBREVIATION': 17, 'I-ABBREVIATION': 18
}

# Load your JSON data
with open("dataset-1-corrected-umayer.json", "r") as f:
    data = json.load(f)

def char_to_token_spans(text, tokens):
    spans = []
    offset = 0
    for token in tokens:
        start = text.find(token, offset)
        end = start + len(token)
        spans.append((start, end))
        offset = end
    return spans

def convert_to_token_labels(annotations):
    results = []
    for idx, (text, ann) in enumerate(annotations):
        tokens = word_tokenize(text)
        spans = char_to_token_spans(text, tokens)
        tags = ['O'] * len(tokens)

        for start_char, end_char, label in ann['entities']:
            bio_prefix = f'B-{label}'
            ioi_prefix = f'I-{label}'
            began = False
            for i, (token_start, token_end) in enumerate(spans):
                if token_end <= start_char:
                    continue
                if token_start >= end_char:
                    break
                tags[i] = bio_prefix if not began else ioi_prefix
                began = True

        tag_ids = [label2id[tag] for tag in tags]
        results.append({
            "id": str(idx),
            "tokens": tokens,
            "ner_tags": tag_ids
        })

    return results

# Convert and print
converted = convert_to_token_labels(data["annotations"])
# print(json.dumps(converted, indent=4))

converted

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\umayer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[{'id': '0',
  'tokens': ['Cellulose',
   '(',
   'C',
   '<',
   'sub',
   '>',
   '6',
   '<',
   '/sub',
   '>',
   'H',
   '<',
   'sub',
   '>',
   '10',
   '<',
   '/sub',
   '>',
   'O',
   '<',
   'sub',
   '>',
   '5',
   '<',
   '/sub',
   '>',
   ')',
   '<',
   'sub',
   '>',
   'n',
   '<',
   '/sub',
   '>',
   'is',
   'one',
   'of',
   'the',
   'most',
   'ubiquitous',
   'organic',
   'polymers',
   'on',
   'the',
   'planet',
   '.',
   'It',
   'is',
   'a',
   'significant',
   'structural',
   'component',
   'of',
   'the',
   'primary',
   'cell',
   'wall',
   'of',
   'green',
   'plants',
   ',',
   'various',
   'forms',
   'of',
   'algae',
   'and',
   'oomycetes',
   '.',
   'It',
   'is',
   'a',
   'polysaccharide',
   'consisting',
   'of',
   'a',
   'linear',
   'chain',
   'of',
   'several',
   'hundred',
   'to',
   'many',
   'thousands',
   'of',
   'Î²',
   '(',
   '1',
   'â†',
   '’',
   '4',
   ')',
   'linked',
   'd-glucose',
   'units',